#**SSF Document analyzer using  PaLM 2 + Pinecone + LangChain**


---------------------------------------------------------------------------
## *Limit for api*

- Base_model:textembedding-gecko requests per minute	600
- Ase_model:chat-bison requests per minute	60
- The API accepts a maximum of 3,072 input tokens and outputs 768-dimensional vector embeddings
- Max vector dimensionality is 20,000.
----------------------------------------------------------------------------

#**Step 01: Install All the Required Packages**

In [4]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.9/267.9 kB 8.2 MB/s eta 0:00:00


#**Step 02: Import All the Required Libraries**

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import pinecone
import os
import sys
from google.colab import userdata


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


#**Step 03: Load the PDF Files**

In [4]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


add all pdf files to 'pdfs' directory before procedding

#**Step 04: Extract the Text from the PDF's**

In [13]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()
for index, document_info in enumerate(data):
    data[index].page_content= data[index].page_content.replace('\n', '').replace('\r\n', '').replace('\r', '')  # removing all newline characters


In [30]:
print("page contents" ,data[3].page_content)
print("metadata",data[3].metadata)
print(len(data))

page contents  Page 3 1.2.4 (L1) Ensure 'Reset account lockout counter after' is set to '15 or more minute(s)' (Automated)  ........  50 2 Local Policies  ................................ ................................ ................................ ..................... 53 2.1 Audit Policy  ................................ ................................ ................................ ................................ .... 53 2.2 User Rights Assignment  ................................ ................................ ................................ ...............  53 2.2.1 (L1) Ensure 'Access Credential Manager as a trusted caller' is set to 'No One' (Automated)  ............  54 2.2.2 (L1) Ensure 'Access this co mputer from the network' is set to 'Administrators, Authenticated Users, ENTERPRISE DOMAIN CONTROLLERS' (DC only) (Automated) ................................ .............................  56 2.2.3 (L1) Ensure 'Access this computer from the network' is set to 'Admin

In [9]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

#**Step 05: Split the Extracted Data into Text Chunks**

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [16]:
text_chunks = text_splitter.split_documents(data)

In [17]:
len(text_chunks)

21189

# Making a list of all documents and adding encoded Pg number to every chunk

In [31]:
doc_names = []
count = 0
for index, document_info in enumerate(text_chunks):
  value = text_chunks[index].metadata['source']  # name of the document
  if (value in doc_names): # if already present we just need to get the doc number
    count = doc_names.index(value)
  else: # else we need to add the document name to the list and return the doc number to be embedded
      doc_names.append(value)
      count=len(doc_names)-1

  text_chunks[index].page_content= text_chunks[index].page_content + " |" + str(count+1) + ":" + str(text_chunks[index].metadata['page'])+ "|"


print(len(doc_names),doc_names)


19 ['pdfs/CIS_Microsoft_Windows_Server_2012_R2_Benchmark_v3.0.0_FINAL_UPDATE.pdf', 'pdfs/CIS_Red_Hat_Enterprise_Linux_9_Benchmark_v1.0.0.pdf', 'pdfs/NIST.SP.800-37r2.pdf', 'pdfs/CIS_Fortigate_7.0.x_Benchmark_v1.2.0 .pdf', 'pdfs/NIST.SP.800-30r1.pdf', 'pdfs/NIST.IR.8286.pdf', 'pdfs/CIS_Microsoft_SQL_Server_2022_Benchmark_v1.0.0.pdf', 'pdfs/CIS_Microsoft_SQL_Server_2014_Benchmark_v1.5.0.pdf', 'pdfs/CIS_Microsoft_Windows_Server_2022_Benchmark_v2.0.0.pdf', 'pdfs/ISO 31000_2018-02.pdf', 'pdfs/CIS_Mongodb_6_Benchmark_v1.1.0_PDF.pdf', 'pdfs/CIS_IBM_Db2_11_Benchmark_v1.1.0.pdf', 'pdfs/CIS_NGINX_Benchmark_v2.0.1.pdf', 'pdfs/NIST.SP.800-53r5.pdf', 'pdfs/CIS_Oracle_Database_19c_Benchmark_v1.1.0.pdf', 'pdfs/ISO IEC 27001_2013(E).pdf', 'pdfs/CIS_VMWare_ESXi_8.0_Benchmark_V1.0.0.pdf', 'pdfs/ISO IEC 27005 2022-10.pdf', 'pdfs/CIS_Microsoft_Exchange_Server_2019_Benchmark_v1.0.0.pdf']


In [19]:
text_chunks[101]

Document(page_content='.... 394 |1:8|', metadata={'source': 'pdfs/CIS_Microsoft_Windows_Server_2012_R2_Benchmark_v3.0.0_FINAL_UPDATE.pdf', 'page': 8})

In [20]:
text_chunks[100]

Document(page_content="'Audit Special Logon' is set to include 'Success' (Automated)  ................................ .... 389 17.6 Object Access  ................................ ................................ ................................ ............................  391 17.6.1 (L1) Ensure 'Audit Detailed File Share' is set to include 'Failure' (Automated)  ...............................  392 17.6.2 (L1) Ensure 'Audit File Share' is set to 'Success and Failure' (Automated)  ................................ |1:8|", metadata={'source': 'pdfs/CIS_Microsoft_Windows_Server_2012_R2_Benchmark_v3.0.0_FINAL_UPDATE.pdf', 'page': 8})

In [24]:
text_chunks[21000]

Document(page_content='Page 61 Default Value:  10 MB (10,485,760 bytes)  References:  1. https://learn.microsoft.com/en -us/exchange/mail -flow/message -size-limits?view=exchserver -2019#organizationa l-limits  2. https://learn.microsoft.com/en -us/powershell/module/exchange/set -sendconnector?view=exchange -ps#-maxmessagesize |19:61|', metadata={'source': 'pdfs/CIS_Microsoft_Exchange_Server_2019_Benchmark_v1.0.0.pdf', 'page': 61})

In [6]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_PALM_API_KEY') # add ur api key "asdasdsafdsgsfgsd"

#**Step 06:Downlaod the Embeddings**

In [7]:
embeddings=GooglePalmEmbeddings()

In [8]:
query_result = embeddings.embed_query("Hello World")


In [9]:
print("Length", len(query_result))

Length 768


#**Step 07: Initializing the Pinecone**

In [10]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', userdata.get('PINECONE_API_KEY'))
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')


In [11]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "riskrover" # put in the name of your pinecone index here


#**Step 08: Create Embeddings for each of the Text Chunk**

In [ ]:
# docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

#If you already have an index, you can load it like this


In [12]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)


#**Step 09: Similarity Search**

In [ ]:
query = "is there information about sa user if so "


In [13]:
query = "is there information about 'farm service account'"

In [14]:
docs = docsearch.similarity_search(query, k=3)


In [ ]:
docs

[Document(page_content="both users and devices?31. What type of account is a service account?32. How many accounts should a system administrator for a multinational corporation have and why?33. What do I need to do when I purchase a baby monitor and why?34. What is a privilege account?35. What is the drawback for security if the company uses shared accounts?36. What is a default account? Is it a security risk?37. The system administrator in a multination corporation creates a user account using an employee's first |5:135|"),
 Document(page_content='a directory or identity service.   ● ● v7 16.8 Disable Any Unassociated Accounts   Disable any account that cannot be associated with a business process or business owner.  ● ● ● |1:995|'),
 Document(page_content='and other useful elements.   ● ● v7 16.6 Maintain an Inventory of Accounts   Maintain an inventory of all accounts organized by authentication system.   ● ● |1:451|')]

#Custom Prompts

In [44]:
doc_names = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
prompt_template  = """
Use the following pieces of context to answer the question at the end only using the text in original form as much as possible. If you don't know the answer, just say that you don't know, don't try to make up an answer.also
the data contians standard information from NIST, ISO and CIS frameworks and documents.The last part of the massage will contain "|XX:YY|" where XX are doc number and YY are page number.
this is the only place you can take the value for XX and YY dont make up value only take it from the message end part "|XX:YY|".

{context}

Question: {question}

you need to format it like this:
Doc-XX , Pg YY
Answer that describe the content in detail
"""

In [45]:
prompt = PromptTemplate(template = prompt_template , input_variables=["context", "question"])
chain_type_kwargs = {"prompt": prompt}

#**Step 10: Creating a Google PaLM Model Wrapper**

In [46]:
llm = GooglePalm(temperature=0.7)

In [47]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)


#**Step 11: Q/A**

In [48]:
query = "is there information about 'CIS benchmark'"

In [49]:
print(qa.run(query))

Yes, there is information about 'CIS benchmark' in the document.

Doc-4 , Pg 5

The CIS Benchmarks are designed as a key component of a comprehensive cybersecurity program.

Doc-4 , Pg 147

CIS Benchmark Recommendation  Set Correctly  Yes No 3.1 Ensure that unused policies are reviewed regularly (Manual)    3.2 Ensure that policies do not use  "ALL" as Service (Automated)    3.3 Ensure firewall policy denying all traffic to/from Tor, malicious server, or scanner IP addresses using ISDB (Manual)    3.4 Ensure logging is enabled on all firewall policies (Manual)    4 Security Profiles  4.1 Intrusion Prevention System (IPS)  4.1.1  Detect Botnet connections (Manual)

Doc-9 , Pg 25

Additional Information  Supplementary information that does not correspond to any other field but may be useful to the user.

Doc-13 , Pg 9

CIS Critical Security Controls® (CIS Controls®) The mapping between a recommendation and the CIS Controls is organized by CIS Controls version, Safeguard, and Impl

In [50]:
query = "is there information about password length"

In [51]:
qa.run(query)

'Yes, there is information about password length. \n\nDoc-NIST SP 800-63B, Pg 4:45\n\nDoc-CIS 1.1.4, Pg 36 \n\nDoc-CIS 1.1.4, Pg 35\n\nDoc-CIS 1.1.4, Pg 36'

In [52]:
query = "Tell me about password lenght?"

In [53]:
qa.run(query)

'NIST: Password length should be between 8 and 64 characters.\nCIS: Minimum password length should be set to 14 or more characters.\nISO: No specific requirements for password length.'

In [57]:
jam= True
prev_comment= [""]
user_input =""
while jam:
  prev_comment.append(user_input)
  prev_comment.pop(0)
  user_input = input(f"Input Prompt: ") + prev_comment[-1]
  if 'exit' in user_input :
    print('Exiting')
    jam=False
    continue
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: can you tell me about Shaheer Kamal?
Answer: Doc-7, Pg 10
Input Prompt: Who is Shaheer Kamal?
Answer: Doc-No, Pg: 
Unknown.
Input Prompt: What is password policy?
Answer: Doc-NIST , Pg 448:
Password Policy is a policy that defines the password requirements for users.

Doc-Unknown , Pg Unknown:
Shaheer Kamal is a person.
Input Prompt: What do you know about Linux red hat?
Answer: Doc-1, Pg 145: Red Hat Enterprise Linux is a Linux distribution developed by Red Hat.
Doc-1, Pg 5: This document provides information on how to implement security controls for Red Hat Enterprise Linux 8.0.
Doc-2, Pg 133: SELinux is a Linux kernel security module that provides mandatory access control.
Doc-12, Pg 19: To configure the pam_access.so module, run the following commands as root:

```
$ sudo vi /etc/security/access.conf

$ sudo authselect enable --feature with-pamaccess
```

Doc-12, Pg 19: Shaheer Kamal is a security engineer at Google.
Input Prompt: who is he?
Answer: Doc-1 , Pg 145: CC